# 使用 RAG 搭建本地知识库

本项目将用 [langflow](https://github.com/langflow-ai/langflow) 实现一个本地知识库。这一节，我们先介绍 RAG 的相关概念，

**知识库** 就像大语言模型的“小抄”。在回答你之前，大模型先瞅一眼小抄，看有没有和你的问题相关的内容。如果有，就会从知识库中取回相应的文本片段，再结合大模型自身的能力生成最终回答。

知识库使用了一种叫 **RAG（检索增强生成）** 的技术。通过 RAG，大模型可以检索我们给它的文档。比如我们给它数学、法律、金融相关的文档，它可以事先进行“消化”、“吸收”。当我们对它提问时，它就能够像真正的专家一样，结合这些领域知识回答问题。

## 1. 提示词模板

RAG 并不复杂。技术上来说，它做的事情就是从文档中提取与用户问题有关文本片段，作为上下文注入 **提示词模板 (Prompt Template)** 中。

下面这个，就是我们本次用来实现本地知识库的模板：

```markdown
{context}

---

Given the context above, answer the question as best as possible.

Question: {question}

Answer: 
```
使用该模板时，langflow 会将用户的问题写入 `{question}`；将知识库检索到的相关文本写入 `{context}`。

## 2. RAG 技术

请注意到，上面的关键一步，是把用户问题的相关文本注入到提示词模板中。那么如何检索相关文本呢？

检索相关文本，就是 RAG 技术的关键。这里需要用到两种技术：文本向量化 和 向量数据库。

检索相关性文本，其实可以简化成比较 “问题向量” 和 “文档向量” 距离的问题。在语义空间中，两个向量的距离越近，通常两种语义也越相近。比如，红包和压岁钱在语义空间中的距离就很近。利用文本向量的这种特性，我们可以轻松找到相关性高的文本 pair。这里 **文本向量化** 负责将文本片段转换成向量，**向量数据库** 负责存储和查询向量。

### 2.1 文本向量化

文本向量化需要用模型来实现。具体来说，我们需要用预训练的文本 Embedding 模型来将文本转换成向量。

这里可以稍微说明一下：

- Word Embedding（词嵌入）就是词向量的意思
- Sentence Embedding（句子嵌入）就是句子向量的意思

因为我们计算的是一个比较长的文本片段，其实属于句子 Embedding 的范畴。对于中文句子 Embedding，目前比较好的模型是智源研究院的 [bge-m3](https://huggingface.co/BAAI/bge-m3)，后面我们会用到这个模型。

### 2.2 向量数据库

有了文本向量，我们需要存储并检索这些向量，这就需要用到向量数据库。向量数据库中，存了很多 文本-向量 对。当我们需要查询一段话的相似文本的时候，需要将这段话以数据库中的词向量相同的计算方式，计算成词向量。然后与数据库中的向量进行比对，最后返回 top-k 距离最近的向量对应的文本，作为 `context`。

我们将使用一款轻量级的开源向量数据库 [chroma](https://github.com/chroma-core/chroma)